In [2]:
%cd /home/silvio/miniconda3/envs/classy3/prg

import linecache
from symspellpy import SymSpell, Verbosity
from rich import print
import inspect
import os
from collections import Counter
import re
import regex
import textstat
from utils.file_utils import file_validate, diy_file_validate

/home/silvio/miniconda3/envs/classy3/prg


In [5]:
CONFIG_DIR = '/home/silvio/miniconda3/envs/classy3/prg/config/'
STOPWORD_ES = 'stopwords_es.txt'
STOPWORD_RED = "stopwords_reddit.txt"
DICTIONARY = "new_dict.txt"
dictionary = os.path.join(CONFIG_DIR, DICTIONARY)
stopwords_files = [os.path.join(CONFIG_DIR, STOPWORD_ES),
                  os.path.join(CONFIG_DIR,STOPWORD_RED)]

In [6]:
class Dictionaries:

    def __init__(self,filename=None, steps=10):
        self._dictionary_file = None
        self._dictionary_steps = None
        self._dictionary_file
        self._auxiliary = []  # List of tuples (filename, position)
        self._dictionary = None
        self._dictionary_num_terms = 0
        self._dictionary_steps_thresholds = []
        self._dictionary(filename, steps)

    def set_dictionary(self, filename=None, steps=10):
        success, message = diy_file_validate(filename)
        if success:
            self._dictionary_file = filename
        else:
            error(0, f"File {filename}: {message}")

        if not steps:
            print("Steps must be greater than 0. Assigned 10 as default.")
            steps = 10
        self._dictionary_steps = steps

    @property
    def dictionary_file(self):
        return self._dictionary_file

    @property
    def dictionary_steps(self):
        return self._dictionary_steps

    @property
    def auxiliary(self):
        return self._auxiliary

    @auxiliary.setter
    def auxiliary(self, value):
        self._auxiliary = value

    @property
    def dictionary(self):
        return self._dictionary

    @dictionary.setter
    def dictionary(self, value):
        self._dictionary = value

    @property
    def dictionary_num_terms(self):
        return self._dictionary_num_terms

    @dictionary_num_terms.setter
    def dictionary_num_terms(self, value):
        self._dictionary_num_terms = value

    @property
    def dictionary_steps_thresholds(self):
        return self._dictionary_steps_thresholds

    @dictionary_steps_thresholds.setter
    def dictionary_steps_thresholds(self, value):
        self._dictionary_steps_thresholds = value


In [7]:
class Stopwords:
    def __init__(self):
        self._stopwords = Counter()
        self._files = []
        _stopwords_set = set()
        
    def load_stopwords(self):
        if not self._files:
            return

        self._stopwords = Counter()

        for filename in self._files:
            success, message = diy_file_validate(filename)
            if not success:
                print(message)
                continue

            with open(filename, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                for line in lines:
                    word = line.strip()
                    if word:
                        self._stopwords.update({word.lower(): 1})

        self._stopwords_set = set(self._stopwords.keys())
        
    @property
    def files(self):
        return self._files

    @files.setter
    def files(self, files_list):
        self._files = files_list
        self.load_stopwords()

    @property
    def stopwords(self):
        return self._stopwords

    @property
    def stopwords_set(self):
        return self._stopwords_set   

In [8]:

class Textmetrics:
    def __init__(self, lang="es", steps=10):
        self._lang = lang
        self._steps = steps
        self._textstats = textstat
        self._textstats.set_lang(lang)
        self._dictionary_file = None
        self._dictionary = None
        self._dictionary_num_terms = 0
        self._dictionary_steps = steps
        self._dictionary_steps_thresholds = []
        self._stopwords = Counter()
        self._stopwords_files = []
        self._misspellings = []
        self._num_words = 0
        self._num_stopwords = 0
        self._text = ""
        self._text_file = ""

    def c(self,line_num):
        """
            When splitting the dictionary for creating the steps,
            it reads a specific line of the dictionary and returns the
            frequency of the associated word.
            SymSpell returns the original word, or the correction, toghether
            with the frequency: this number will be usde to classify the difficulty
            of a word.
        """
        line = linecache.getline(self._dictionary_file, line_num)
        if line:
            words = line.split()
            if len(words) >= 2:
                second_word = words[1]
                try:
                    second_word_as_int = int(second_word)
                except ValueError:
                    print("Second word is not an integer.")
            else:
                print("Line doesn't contain at least two words.")
        return second_word_as_int


    def stat(self, function_name):
        """
            applies the textstat routine specified in function_name
            to self._textand and returns the result
            Is like calling self._textstats.function_name(self._text)
            but allows to decouple the two objects
        """
        # Check if the function_name exists in textstat
        if hasattr(self._textstats, function_name) and callable(getattr(self._textstats, function_name)):
            # Call the function dynamically with self._text as a parameter
            func = getattr(self._textstats, function_name)
            result = func(self._text)
            return result
        else:
            # Handle the case where the function doesn't exist
            return "Function not found"

    @property
    def stopwords_files(self):
        return self._stopwords_files

    @stopwords_files.setter
    def stopwords_files(self, files_list):
        """
            Loads stopwords from as many file as are contained in files_list.
            The purpose is to allow stopwords for specifica fields to be loaded.
            For example, if it were a medical supplement to the standard stopwords,
            it would contain words like, hospital, IV, transfusion, ER, OR,  nurse, injection, etc
            NOTE: we are talking about stopwords, that is, words that are NOT considered imprescindible unless used by some special dictionaries. Special dictionaries should delete from the stopwords its own words.
            to read and understand a medical text, but they could be important
            to analize the content.
        """
        self._stopwords_files = files_list
        self._stopwords = Counter()
        for file_path in self._stopwords_files:
            try:
                # Open and read the file
                with open(file_path, 'r', encoding='utf-8') as file:
                    lines = file.readlines()
                    # Update the Counter with words (stripped of whitespace and newline characters)
                    for line in lines:
                        word = line.strip()
                        if word:
                            self._stopwords.update([word.lower()])
            except FileNotFoundError:
                print(f"File not found: {file_path}")
            except Exception as e:
                print(f"An error occurred while reading {file_path}: {str(e)}")

        # self._stopwords = stopwords


    @property
    def stopwords(self):
        return self._stopwords


    """
    _text contains the string for which we want statistics in general
    """
    @property
    def text(self):
        return self._text

    @text.setter
    def text(self, txt):
        self._text = txt
        self._misspellings = []
        self._num_words = 0

    @property
    def text_file(self):
        return self._text_file

    @text_file.setter
    def text_file(self, file_path):
        self._text_file = file_path
        try:
            with open(file_path, 'r') as file:
                self._text(file.read())
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred while reading {file_path}: {str(e)}")



    """
    _dictionary_steps containes the blocks into which we want to split the dictionary.
    As an example, let's say that we divide a dictionay in 10 parts with the purpose
    of assessing students from first to tenth grade. In first grade the should be able to
    understand words extracted from the first part of the dictionry, and the same applies
    to students of the fifth grade.
    """
    @property
    def dictionary_steps(self):
        return self._dictionary_steps

    @dictionary_steps.setter
    def dictionary_steps(self, number_of_steps):
        self._dictionary_steps = number_of_steps

    #TODO: add the possibility to merge two dictionaries
    #TODO: add the possibility to insert specialized dictionaries at a given position
    @property
    def dictionary_file(self):
        return self._dictionary

    @dictionary_file.setter
    def dictionary_file(self, file):
        self._dictionary_file = file
        with open(self._dictionary_file, "rbU") as f:
            self._dictionary_num_terms = sum(1 for _ in f)
        self._dictionary_block_size = self._dictionary_num_terms //  self._dictionary_steps

        line_numbers = []
        thresholds = []
        for i in range(1, self._dictionary_steps):
            line_numbers.append(self._steps * i)
            line_num = self._dictionary_num_terms //  self._dictionary_steps * i
            self._dictionary_steps_thresholds.append(self.get_line_frequency(line_num))
        self._dictionary_steps_thresholds.append(self.get_line_frequency(self._dictionary_num_terms))

In [9]:
tm = Textmetrics(lang='es')
tm.stopwords_files = stopwords_files
tm.dictionary_file = dictionary

FileNotFoundError: [Errno 2] No such file or directory: '/home/silvio/miniconda3/envs/classy3/prg/config/new_dict.txt'

In [ ]:
s = """
Hace 2 años conocí a una chica por medio de un amigo, hablamos y tiempo después nos hicimos novios.

La relación era algo estable, teníamos defectos como cada persona, pero a diferencia yo los traba de resolver y cambiar mi actitud, mientras ella solo se dedicaba a enojarse y desquitarse conmigo aun cuando yo no tenía nada que ver, se enojaba con su familia y se desquitaba conmigo, se enojaba en la universidad con sus compañeros o amigos y se desquitaba conmigo, le explique que eso me hacía sentir mal y en cierto modo me hacía sentir inseguro, traté que en el tiempo de relación ella cambiara pero nunca lo hizo, y siempre me decía que era su actitud y debía aguantarle todos sus berrinches, pero cuando llegamos al año y 4 meses las cosas se volvieron densas, teníamos problemas, discusiones, peleas, etc. En mi mente pasaba que ya terminó la etapa de ilusión o enamoramiento como lo llaman los demás, el hecho es que ella buscaba que yo le solucione los problemas y si no lo hacía, la historia de siempre, peleas.

Al año y 6 meses tuvimos una discusión fuerte a la cual ella decidió terminar conmigo, ¿la causa fue que no le daba soluciones a sus problemas, como quería que solucione sus problemas si no podía resolver los que yo tenía? ¿Terminó conmigo haciéndome sentir que fui el malo de la relación, aun cuando yo estuve ahi cuando todos le dieron la espalda, la apoyé aun cuando su madre la trataba mal, enserio fui el malo de la relación? Me enfadé mucho con ella pero no le dije nada, simplemente me ahorré la discusión, tiempo despues, al volver a clases de nuevo nos encontramos y quedamos en un "acuerdo" donde nadie podría salir con nadie por el bien de la "relación" eso me hacía sentir incomodo porque es lo que ma detestaba en alguien, pretender que solo con un "acuerdo" solucionas las cosas, no pude decir nada porque su actitud hacia alguien que no cumple sus caprichos es algo muy inmaduro. Ella dijo que en ese tiempo va a cambiar de actitud, pero no fue asi, la mayoria del tiempo se pasaba diciendo tonterías al frente mio, cosas que en cierta cosa me hieren, pero como ya no eramos novios lo dejé pasar.

En 1 mes tuvimos una pequeña discusión solo porque me acerque a saludar a unas amigas, yo no vi el problema pero ella buscó hasta los likes en cada foto de mis amistades en FB, hasta que me hizo problema de que se sentia insegura y del porque debo estar reaccionando a las fotos de otras chicas, mi respuesta solo fue: ya no estamos en una relación, asi que puedo hacerlo. Ella solo empezó a llorar y decir que no es lo que ella quiere en un "acuerdo". Así pasamos hablando durante 1 hora, donde ella me atacaba en cierta forma y yo solo me quedaba callado hasta que se me salio de un problema que tuve gracias a ella donde involucró antidepresivos por lo mal que me solia hacer sentir cuando se desquitaba confmigo. Por qué no dije hasta ahi quedan las cosas y ya no hay acuerdo? no se en que estaba pensando, creo porque aun sentia algo por ella. Poco a poco me alejaba de ella por mi paz mental, pero no era suficiente

3 meses después pasó lo mismo, pero en este caso fue un comentario que le hice a una chica, fue lo mismo que la anterior vez, tachandome de infiel y que estoy rompiendo su "acuerdo", maldita sea hubiera dicho que ya no quiero seguir con eso y alejarme de ella.

Me juntaba muy bien con su grupo de amigos, al tal punto de compartir algunas cosas privadas mias, ahi fue mi error. Conocí a una chica por internet y nos llevabamos bien, era de otra ciudad y bueno creo que nos empezamos a gustar, esta chica quería venir a la ciudad donde vivo, la verdad no sabia a donde llevarla porque no tenia dinero, nuestras intenciones no solo era pasear, si no de hacer otras cosas. Le dije a uno de ese grupo de amigos que lo llamaremos H, que si no podría prestarme un cuarto, pero dijo que no.

3 semanas despues tuvimos que hacer un proyecto de una materia donde estabamos su grupo de amigos haciendo y ella habia faltado por asuntos de trabajo, salimos a comprar con un amigo que me junto muy bien y al regresar este chico al que le dije si podia prestarme un cuarto, nos salio con la noticia que le iban a sacar del grupo por no venir, este se le comento a la chica y se armó una pelea entre ella y nosotros. Le explique que la idea no fue mia, que a mi solo me avisaron. Al dia siguiente ella asistio para realizar este trabajo y si tocaba comprar algo ella debia poner el dinero que no pudo dar el dia anterior, salio con este chico H, este le habia dicho que yo era el cabecilla de sacarla del grupo. Cuando terminamos y fuimos a tomar algo, ella me confrontó diciendo que porque tuve esa idea, los demas chicos se quedaron con cara de sorprendidos, le dije que no que fue este chico H, en fin, este chico es muy inmaduro para su edad y el dia siguiente le mostró el chat donde le habia preguntado eso.

Ella despues de clases me confrontó diciendome que quien era, le expliqué y a lo cual ella procedio a agredirme, es la segunda vez que lo hacía, en medio de los golpes, aruñasos, etc. me nacía ese deseo de devolverle un golpe, pero por razones divinas me contuve, sali corriendo de ahi y por suerte me encontré con un ingeniero a lo cual le empecé a hacer la conversa, ella estuvo todo el tiempo ahi esperando, al acabar la conversa me llevo a otro lugar donde me dijo que soy una mierda de persona, que es la primera vez que alguien le hace sentir asi, que espera que me vaya mal en todo y cosas asi.

Lo unico que le dije fue: Yo te fui sincero y honesto, nunca te fallé durante la realción, estoy orgulloso de eso, de aqui a lo que yo haga cuando tu terminaste conmigo es problema mio. Ella lloró, pataleó, poniendose en un lugar donde el puesto le quedaba grande, el sentirse madura, el sentirse que siempre ella tiene la razon. Yo solo tomaba distancia para que no me pegue. Terminó su discurso, fui a casa, le conté todo a mi madre, lloré un poco y ella me calmó diciendo que lo que dije estuvo bien, no tengo responsabilidad con ella despues de haber terminado. me dijo qye podia hacer lo que se me diera la gana.

El siguiente dia siguiente fui a la universidad, no saludé con sus amigos porque me dijo que ya no quiere que me junte con ellos, me senté en la aprte de atras, 2 de ellos se acercaron y me dijeron que ella no es quien para obligarlos a juntarse o no conmigo, que seguiriamos siendo amigos y no importa que cosas diga ella de mi, no cambiaran de opinion y que ella al agredirme estruvo demasiado mal y el contar orgullosamente de que me alzo la mano estuvo peor. Juro que quería llorar porque al fin alguien esta de mi lado y me entiende. Me dijeron que el chico H le contó todo porque le dije que el tuvo la idea de sacarla del grupo de trabajo y que le valia la "amistad que teniamos", me enfadé mucho con este chico a lo cual solo les dije que si me viene a molestar o algo que en verdad me enfade me desquitaria de todo lo que lla me dijo

Hoy fue la exposicion de dicho trabajo, ella lanzaba comentarios a diestra y siniestra de mi, solo me reía hasta que le dije a uno de los chicos que me vino a ver ese día, le dije que si seguia diciendo cosas de mi, el chico H no saldría bien ese día.

saludé con mis amigas del otro curso, prestandole cosas, riendo, bromeando, tal vez un comentario picarón. al finalizar la epxosicion fuimos a tomar algo, ella al despedirse, enojada me dice que que onda con las "amiguitas", que la respete, mi respuesta solo fue el decir: respetar que, tu misma lo dijiste.

Solo se que si de nuevo empieza a lanzar comentarios que me ofendan, actos que me afecten o violencia fisica, me desquitaría con este chico H, no por avisarla sobre lo que le dije, si no por lo hipócrita que es conmigo, con ella y con sus amigos.
"""


In [ ]:
tm.text = s


In [ ]:
print(tm.stat("fernandez_huerta"))


In [ ]:
print(tm._dictionary_steps_thresholds)

In [ ]:
print(vars(tm))

In [ ]:
spanish_text = "— Siglos que no se te ve, señor editorialista — dice Norwin—. ¿Estás más contento en la página editorial que en locales?"

In [ ]:
import re

def count_words(text):
    # Use the \w+ pattern to match word characters (letters, numbers, underscores) in Spanish.
    # This pattern may need to be adjusted based on your specific requirements for word boundaries.
    word_pattern = r'\w+'

    # Find all matches of the word pattern in the text.
    words = re.findall(word_pattern, text, re.UNICODE)

    # Return the count of words.
    return len(words)

# Example usage:
word_count = count_words(spanish_text)
print(f"Word count: {word_count}")


In [ ]:
def count_characters(text):
    # Use the Python built-in function len() to count characters in a Unicode string.
    # This method works for various languages, including Spanish, English, and Italian.
    character_count = len(text)
    
    return character_count

# Example usage:
multilingual_text = "Texto en español,."
character_count = count_characters(multilingual_text)
print(f"Character count: {character_count}")


In [ ]:
def count_letters(text):
    # Use the pattern \p{L} to match any Unicode letter.
    letter_pattern = r'\p{L}'

    # Find all matches of the letter pattern in the text.
    letters = regex.findall(letter_pattern, text, re.UNICODE)

    # Return the count of letters.
    return len(letters)


# Example usage:
multilingual_text = "Texto en español, English text, Testo in italiano."
letter_count = count_letters(multilingual_text)
print(f"Letter count: {letter_count}")
print(len(multilingual_text))
# In this code, we use the \p{L} pattern to match any Unicode letter, which works for a variety of languages. The re.UNICODE flag ensures that it properly handles Unicode characters. This function counts the number of letters in the input string without relying on language-specific rules.
